In [2]:
# %pip install scikit-surprise==1.1.0 # pickle

In [3]:
from surprise import accuracy, Reader, Dataset, SVD
from surprise.model_selection import cross_validate

In [4]:
import pandas as pd
import pathlib
DATA_DIR = pathlib.Path().resolve().parent / "data"
print(DATA_DIR)
DATA_DIR.exists()

/home/louis/astraZenith/backend_recommender/recommender/src/data


True

In [5]:
dataset = DATA_DIR / 'ratings_small.csv'
dataset.exists()

True

In [6]:
df = pd.read_csv(dataset)
df['rating'].dropna(inplace=True) # drop NaNs in rating column
df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [7]:
df.rating.max(), df.rating.min()

(5.0, 0.5)

In [8]:
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)

In [9]:
algo = SVD(verbose=True, n_epochs=20)
cross_validate(algo, data, measures=['RMSE', "MAE"], cv=4, verbose=True)

Processing epoch 0


Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processin

{'test_rmse': array([0.89740793, 0.90357971, 0.90430958, 0.88786694]),
 'test_mae': array([0.69094387, 0.69667346, 0.69693541, 0.68393348]),
 'fit_time': (1.6577746868133545,
  1.63899827003479,
  1.696925401687622,
  1.620718002319336),
 'test_time': (0.1096184253692627,
  0.06035113334655762,
  0.06104588508605957,
  0.09860754013061523)}

In [10]:
trainset = data.build_full_trainset()
algo.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19


In [11]:
testset = trainset.build_testset()
predictions = algo.test(testset)
# RMSE should be low as we are biased
accuracy.rmse(predictions, verbose=True)
accuracy.mae(predictions, verbose=True)

RMSE: 0.6439
MAE:  0.4984


0.4984139987887754

In [12]:
sample_row = df.sample(n=5)
userId   = sample_row['userId'].values[0]
movieId = sample_row['movieId'].values[0]
rating   = sample_row['rating'].values[0]
print(userId, movieId, rating)

547 31109 3.0


In [13]:
pred = algo.predict(uid=userId, iid=movieId)
pred.est

3.54642531980884

In [14]:
import pickle

In [15]:
algo_data = {"model": algo}
with open('model.pkl', 'wb') as f:
    pickle.dump(algo_data, f)

In [16]:
model_algo= None
with open('model.pkl', 'rb') as f:
    model_data_loaded = pickle.load(f)
    model_algo = model_data_loaded.get('model')

In [17]:
model_algo.predict(uid=userId, iid=movieId).est

3.54642531980884

In [18]:
sample_rows = df.sample(n=10).to_dict('records')
for row in sample_rows:
    userId = row['userId']
    movieId = row['movieId']
    pred = model_algo.predict(uid=userId, iid=movieId).est
    print(userId, movieId, pred)

547 8853 3.1697188230063524
242 260 5
472 412 3.2791388860442563
102 296 4.614277563473047
78 26614 4.495313709982936
405 2919 3.6712583645012824
516 586 3.3013180398060666
202 3766 3.807780837487288
455 2194 3.7358713246911384
596 4344 3.15234233386346
